In [7]:
## 导入会使用到的库
%config InlineBackend.figure_format = 'retina'
%matplotlib inline    
import seaborn as sns 
sns.set(font= "simhei",style="whitegrid",font_scale=1.6)
import matplotlib
matplotlib.rcParams['axes.unicode_minus']=False # 解决坐标轴的负号显示问题
## 导入需要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px

## 忽略提醒
import warnings
warnings.filterwarnings("ignore")

In [8]:
df1 = pd.read_excel('数据\\附件1.xlsx',sheet_name=0)
df2 = pd.read_excel('数据\\附件1.xlsx',sheet_name=1)
df3 = pd.read_excel('数据\\附件2.xlsx',sheet_name=0)
df4 = pd.read_excel('数据\\附件2.xlsx',sheet_name=1)

#对附件1的sheet2做处理
ls = []
for name,typ,soils in zip(df2.作物名称,df2.作物类型,df2.种植耕地):
    for soil in soils.split('\n\n'):
        ls.append([name,typ,soil])
d = pd.DataFrame(ls,columns=['作物名称','作物类型','种植耕地'])

ls1 = []
ls2 = []
for i in d.种植耕地:
    if '季' in i:
        ls1.append(i.split(' ')[0])
        ls2.append(i.split(' ')[-1].replace('\n',''))
    else:
        ls1.append(i.replace('\n',''))
        ls2.append('单季')
d['种植耕地'] = ls1

#将字段进行差分
ls1 = []
ls2 = []
for i in d.作物类型:
    if '豆类' in i:
        ls1.append(i.split('（')[0])
        ls2.append('是')
    else:
        ls1.append(i)
        ls2.append('不是')
d['作物类型'] = ls1
d['是否豆类'] = ls2
df2 = d

#同样的逻辑，将附件2的sheet1处理
df3.columns = ['地块名称', '作物编号', '作物名称', '作物类型', '种植面积/亩', '种植季次']
df3 = pd.merge(df1,df3,how='right',on='地块名称')

df3 = pd.DataFrame(df3.groupby(['地块类型','作物编号']).sum()[['种植面积/亩']])
ls1 = []
ls2 = []
for i,o in df3.index:
    ls1.append(i)
    ls2.append(o)
df3['地块类型'] = ls1
df3['作物编号'] = ls2
df3.reset_index(drop=True,inplace=True)

#数据合并
df4 = pd.merge(df4,df3,how='left',on=['地块类型','作物编号']).fillna(0)
df4['地块类型'] = [i.replace(' ','') for i in df4.地块类型]

df2.columns = ['作物名称', '作物类型', '地块类型', '是否豆类']

result = pd.merge(df4,df2,how='left',on=['作物名称', '地块类型'])
result.to_excel('处理后数据.xlsx',index=False)

In [9]:
ls = []
for i in result['销售单价/(元/斤)']:
    n = float(i.split('-')[0])
    m = float(i.split('-')[1])
    ls.append((n+m)/2)
result['平均销售单价'] = ls

result['总销量'] = result['亩产量/斤']*result['种植面积/亩']
result['总成本'] = result['平均销售单价']*result['种植面积/亩']
result['总盈利'] = result['总销量']*result['平均销售单价']-result['总成本']

df = result.copy()

In [10]:
for index,i in enumerate(df['地块类型'].unique()):
    df.loc[df.地块类型==i,'地块类型'] = index
df = df.sort_values(by=['地块类型','作物编号'])
for index,i in enumerate(df['作物类型'].unique()):
    df.loc[df.作物类型==i,'作物类型'] = index
for index,i in enumerate(df1['地块类型'].unique()):
    df1.loc[df1.地块类型==i,'地块类型'] = index
df['种植产量'] = df['亩产量/斤']*df['种植面积/亩']

for index,i in enumerate(df['作物类型'].unique()):
    df.loc[df.作物类型==i,'作物类型'] = index

ls1 = []
ls2 = []
ls3 = []
ls4 = []
ls5 = []
ls6 = []
for index,i in enumerate(zip(df.作物类型,df.种植季次,df.地块类型,df.作物名称)):
    if i[0] == 0 and i[3] != '水稻':
        ls1.append(index)
    elif i[3] == '水稻':
        ls2.append(index)
    elif (i[2] == 4 or i[2] == 5) and i[0] != 2:
        ls6.append(index)
    elif i[0] == 2:
        ls5.append(index)
    elif i[1] == '第一季':
        ls3.append(index)
    elif i[1] == '第二季':
        ls4.append(index)
bean = []
for index,i in enumerate(df['是否豆类']):
    if i =='是':
        bean.append(index)
        
def find_indices(lst, element):
    return [index for index, value in enumerate(lst) if value == element]
#多余的作物需要花费的成本
def function(sales_min1,sales_min2,sales_min3):
    for i in sales_min.keys():
        total = 0
        index_ls = find_indices(land_types, i)

        total_product1 = sum([yields[j]*areas[i]*x[i][j][o]*0.25 for i in range(num_plots) for j in index_ls for o in range(2)])
        total_product2 = sum([yields[j]*areas[i]*x[i][j][o]*0.25 for i in range(num_plots) for j in index_ls for o in range(2,4)])
        total_product3 = sum([yields[j]*areas[i]*x[i][j][o]*0.25 for i in range(num_plots) for j in index_ls for o in range(4,6)])

        if total_product1 <= sales_min1[i]:
            pass
        else:
            total += (total_product-sales_min1[i])*prices[index_ls[0]]
        if total_product2 <= sales_min2[i]:
            pass
        else:
            total += (total_product-sales_min2[i])*prices[index_ls[0]]
            
        if total_product3 <= sales_min3[i]:
            pass
        else:
            total += (total_product-sales_min3[i])*prices[index_ls[0]]
    return total

#### 问题 2 根据经验，小麦和玉米未来的预期销售量有增长的趋势，平均年增长率介于5%-10%之间，其他农作物未来每年的预期销售量相对于 2023 年大约有±5%的变化。农作物的亩产量往往会受气候等因素的影响，每年会有±10%的变化。因受市场条件影响，农作物的种植成本平均每年增长5%左右。粮食类作物的销售价格基本稳定；蔬菜类作物的销售价格有增长的趋势，平均每年增长5%左右。食用菌的销售价格稳中有降，大约每年可下降1%~5%，特别是羊肚菌的销售价格每年下降幅度为5%。

#### 对于未来n年，我们只需要优化最近三年的内容，只要保证每三年，每个土地都种植豆类，那么即可保证土地是营养的，并且最终的目标模型以利润最大化目标进行度量，所以未来n年，我们只需要加入这种随机扰动，依次填写优化结果即可

### 情况一：超过部分滞销，造成浪费

In [56]:
#预定义，防止回到原始数据的情况
sales = (df.groupby('作物编号').sum()['种植产量']*1)
yields = df['亩产量/斤'].tolist()  # 产量列表
cos = []
for o in range(num_plots):
    cos.append(df['种植成本/(元/亩)'].tolist())
costs = cos # 成本矩阵

prices = (df['亩产量/斤']*df['平均销售单价']).tolist()  # 价格列表


In [57]:
def group():#加入随机销量成长率  
    global sales,sales_min1,sales_min2,sales_min3,yields,costs,price
    for i in sales.keys():
        if i == 6 or i == 7:
            sales[6] = sales[6]*(1+random.uniform(0.05,0.1))
            sales[7] = sales[7]*(1+random.uniform(0.05,0.1))
        else:
            sales[i] = sales[i]*(1+random.uniform(-0.05,0.05))
    sales_min1 = (sales*0.9).to_dict()
    #第二年
    for i in sales.keys():
        if i == 6 or i == 7:
            sales[6] = sales[6]*(1+random.uniform(0.05,0.1))
            sales[7] = sales[7]*(1+random.uniform(0.05,0.1))
        else:
            sales[i] = sales[i]*(1+random.uniform(-0.05,0.05))
    sales_min2 = (sales*0.9).to_dict()
    #第三年
    for i in sales.keys():
        if i == 6 or i == 7:
            sales[6] = sales[6]*(1+random.uniform(0.05,0.1))
            sales[7] = sales[7]*(1+random.uniform(0.05,0.1))
        else:
            sales[i] = sales[i]*(1+random.uniform(-0.05,0.05))
    sales_min3 = (sales*0.9).to_dict()

    #产量
    yields = [i*(1+random.uniform(-0.1,0.1)) for i in yields]

    #成本
    costs = [(np.array(costs[0])*1.05).tolist() for o in range(num_plots) ]

    for index,i in enumerate(df['作物类型'].values):
        if i == 1:
            prices[index] = prices[index]*1.05
        elif i == 2:
            prices[index] = prices[index]*(1+random.uniform(0.01,0.05))


In [58]:
import pulp
import random


# 54 块地，每块地都有特定的类型 (0: 平旱地, 1: 梯田, 2: 山坡, 3: 水浇地, 4: 普通大棚, 5: 智慧大棚)
num_plots = 54
num_crops = len(df['作物编号'].tolist())  # 125 种属于不同地块不同季节的作物
land_types =  df['作物编号'].tolist()# 0: 平旱地, 1: 梯田, 2: 山坡, 3: 水浇地, 4: 普通大棚, 5: 智慧大棚
plot_type = df1['地块类型'].tolist()  # 每个地块对应的地块类型

# 定义变量 x[i][j][t]，表示在第i块地是否种植第j种作物，t 表示第 t 季节
x = pulp.LpVariable.dicts("x", (range(num_plots), range(num_crops), range(6)), cat="Binary")

areas = df1['地块面积/亩'].tolist()  # 地块面积

# 定义作物集
grain_crops = ls1  # 粮食作物集
rice_crop = ls2  # 水稻
vegetable1_crops = ls3  # 第一季蔬菜集
vegetable2_crops = ls4  # 第二季蔬菜集
fungi_crop = ls5  # 食用菌
vegetable_crops = ls6  # 智慧大棚、普通大棚蔬菜集
total_set = ls1 + ls2 + ls3 + ls4 + ls5 + ls6
ex_grain_crops = [item for item in total_set if item not in grain_crops]  # 除粮食作物集剩余元素
ex_rice_crop = [item for item in total_set if item not in rice_crop]  # 水稻..
ex_vegetable1_crops = [item for item in total_set if item not in vegetable1_crops]  # 第一季蔬菜集
ex_vegetable2_crops = [item for item in total_set if item not in vegetable2_crops]  # 第二季蔬菜集
ex_fungi_crop = [item for item in total_set if item not in fungi_crop]  # 食用菌
ex_vegetable_crops = [item for item in total_set if item not in vegetable_crops]  # 智慧大棚、普通大棚蔬菜集

bean_crops = bean #大豆索引
time = 6

ex_grain_crops = [item for item in total_set if item not in grain_crops]  # 除粮食作物集剩余元素

def opt():
    # 定义问题
    model = pulp.LpProblem("Land_Type_Optimization", pulp.LpMaximize)
    # 目标函数：利润最大化
    model += pulp.lpSum([(prices[j] * yields[j] - costs[i][j]) * areas[i]*0.25 * x[i][j][t] 
                         for i in range(num_plots) for j in range(num_crops) for t in range(time)]) - function(sales_min1,sales_min2,sales_min3)*0.5 
                        #策略一


    # 地块类型约束
    for i in range(num_plots):
        if plot_type[i] in [0, 1, 2]:  # 平旱地、梯田、山坡
            for o in [0,2,4]:
                model += pulp.lpSum([x[i][j][o] for j in grain_crops]) ==4 # 每块地只能种一种作物
                model += pulp.lpSum([x[i][j][o+1] for j in range(num_crops)]) == 0  # 只能种一季，第二季不种


            model += pulp.lpSum([x[i][j][0]+x[i][j][2]+x[i][j][4] for j in bean_crops]) == 1 # 三年内必须有一次豆类

            for o in range(time):
                model += pulp.lpSum([x[i][j][o] for j in ex_grain_crops]) == 0  # 其余不能种

        elif plot_type[i] == 3:  # 水浇地
            for o in [0,2,4]:
                model += pulp.lpSum([x[i][45][o]]) + pulp.lpSum([x[i][j][o] for j in vegetable1_crops]) == 1  # 一季水稻或第一季蔬菜
                model += pulp.lpSum([x[i][j][o] for j in vegetable1_crops]) == pulp.lpSum([x[i][j][o+1] for j in vegetable2_crops])
                model += pulp.lpSum([x[i][45][o+1]]) == 0

                #model += pulp.lpSum([x[i][j][o+1] for j in vegetable2_crops]) + pulp.lpSum([x[i][j][o] for j in rice_crop]) == 1  # 如果第一季种菜，第二季只能种特定蔬菜

                model += pulp.lpSum([x[i][j][o+1] for j in ex_vegetable2_crops]) == 0  # 其余的不能种
                model += pulp.lpSum([x[i][j][o] for j in ex_vegetable1_crops]) == 0  # 其余的不能种



            model += pulp.lpSum([x[i][j][o] for j in bean_crops for o in range(time)]) == 1 # 三年内必须有一次豆类

        elif plot_type[i] == 4:  # 普通大棚
            for o in [0,2,4]:
                model += pulp.lpSum([x[i][j][o] for j in vegetable_crops]) == 4  # 一季蔬菜
                model += pulp.lpSum([x[i][j][o+1] for j in fungi_crop]) ==2  # 一季食用菌



                model += pulp.lpSum([x[i][j][o] for j in ex_vegetable_crops]) == 0  # 其余不能种
                model += pulp.lpSum([x[i][j][o+1] for j in ex_fungi_crop]) == 0  # 其余不能种

            model += pulp.lpSum([x[i][j][0]+x[i][j][2]+x[i][j][4] for j in bean_crops]) == 1 # 三年内必须有一次豆类

        elif plot_type[i] == 5:  # 智慧大棚
            for o in range(time):
                model += pulp.lpSum([x[i][j][o] for j in vegetable_crops]) == 4  # 一季蔬菜
                model += pulp.lpSum([x[i][j][o] for j in ex_vegetable_crops]) == 0  # 其余不能种



            model += pulp.lpSum([x[i][j][o] for j in bean_crops for o in range(time)]) == 1 # 三年内必须有一次豆类
    for j in range(num_crops):
        model += pulp.lpSum([x[i][j][o] for i in range(num_plots) for o in range(time)]) <= 2
        model += pulp.lpSum([x[i][j][o] for i in range(num_plots) if plot_type[i] == 4 for o in range(time)]) >= 2

    model.solve()

    # 输出结果
    for i in range(num_plots):
        for j in range(num_crops):
            for t in range(6):
                if pulp.value(x[i][j][t]) == 1:
                    print(f"Plot {i} should plant crop {j} in season {t}")
    return x

#制作结果图表
a = df2[['作物名称','作物类型']].drop_duplicates()
dic = {}
for i,o in zip(df2.作物名称,df2.作物类型):
    dic[i] = o               
reward = pd.DataFrame(index=df['作物名称'].unique())
reward['作物类型'] = [dic[i] for i in reward.index]

for wh in range(3):
    group()
    x = opt()
    for t in range(6):
        result = []
        product = []
        for i in range(num_plots):
            tem = []
            product_tem = []
            for j in range(num_crops):
                if pulp.value(x[i][j][t]) == 1:
                    tem.append(df1['地块面积/亩'].values[i])
                    product_tem.append(df1['地块面积/亩'].values[i]*df['亩产量/斤'].values[j]*0.25)
                else:
                    tem.append(0)
                    product_tem.append(0)
                    
            result.append(tem)
            product.append(product_tem)
        result = pd.DataFrame(result,index=df1['地块名称'].values,columns=df['作物编号'].values).T
        result = result.groupby(result.index).sum().T
        result.columns = df['作物名称'].unique()
        result.to_excel(f'问题2\\result1_1_{t%2+1}季(第{2023+t//2+1+wh*3}年).xlsx')
        
        product = pd.DataFrame(product,index=df1['地块名称'].values,columns=df['作物编号'].values).T
        product = product.groupby(product.index).sum().T
        product.columns = df['作物名称'].unique()
        reward[f'{2023+t//2+1+wh*3}年{t%2+1}季产量（斤）'] = product.sum()
reward

Plot 0 should plant crop 1 in season 2
Plot 0 should plant crop 6 in season 0
Plot 0 should plant crop 6 in season 2
Plot 0 should plant crop 6 in season 4
Plot 0 should plant crop 21 in season 4
Plot 0 should plant crop 24 in season 0
Plot 0 should plant crop 24 in season 2
Plot 0 should plant crop 24 in season 4
Plot 0 should plant crop 26 in season 0
Plot 0 should plant crop 26 in season 2
Plot 0 should plant crop 41 in season 0
Plot 0 should plant crop 41 in season 4
Plot 1 should plant crop 4 in season 0
Plot 1 should plant crop 6 in season 0
Plot 1 should plant crop 6 in season 2
Plot 1 should plant crop 6 in season 4
Plot 1 should plant crop 7 in season 0
Plot 1 should plant crop 7 in season 2
Plot 1 should plant crop 7 in season 4
Plot 1 should plant crop 24 in season 0
Plot 1 should plant crop 24 in season 2
Plot 1 should plant crop 24 in season 4
Plot 1 should plant crop 35 in season 2
Plot 1 should plant crop 35 in season 4
Plot 2 should plant crop 6 in season 0
Plot 2 shoul

Plot 0 should plant crop 1 in season 0
Plot 0 should plant crop 5 in season 2
Plot 0 should plant crop 5 in season 4
Plot 0 should plant crop 6 in season 0
Plot 0 should plant crop 6 in season 4
Plot 0 should plant crop 9 in season 2
Plot 0 should plant crop 21 in season 2
Plot 0 should plant crop 21 in season 4
Plot 0 should plant crop 23 in season 0
Plot 0 should plant crop 23 in season 2
Plot 0 should plant crop 26 in season 0
Plot 1 should plant crop 4 in season 0
Plot 1 should plant crop 7 in season 0
Plot 1 should plant crop 7 in season 2
Plot 1 should plant crop 14 in season 2
Plot 1 should plant crop 14 in season 4
Plot 1 should plant crop 22 in season 0
Plot 1 should plant crop 22 in season 4
Plot 1 should plant crop 28 in season 0
Plot 1 should plant crop 29 in season 2
Plot 1 should plant crop 29 in season 4
Plot 1 should plant crop 37 in season 2
Plot 1 should plant crop 37 in season 4
Plot 2 should plant crop 17 in season 0
Plot 3 should plant crop 9 in season 0
Plot 3 sho

Plot 0 should plant crop 1 in season 2
Plot 0 should plant crop 5 in season 2
Plot 0 should plant crop 5 in season 4
Plot 0 should plant crop 6 in season 0
Plot 0 should plant crop 6 in season 2
Plot 0 should plant crop 21 in season 0
Plot 0 should plant crop 21 in season 4
Plot 0 should plant crop 26 in season 0
Plot 0 should plant crop 26 in season 2
Plot 0 should plant crop 26 in season 4
Plot 0 should plant crop 41 in season 0
Plot 0 should plant crop 41 in season 4
Plot 1 should plant crop 14 in season 2
Plot 1 should plant crop 14 in season 4
Plot 1 should plant crop 22 in season 2
Plot 1 should plant crop 22 in season 4
Plot 1 should plant crop 26 in season 0
Plot 1 should plant crop 26 in season 2
Plot 1 should plant crop 26 in season 4
Plot 1 should plant crop 28 in season 0
Plot 1 should plant crop 29 in season 2
Plot 1 should plant crop 31 in season 0
Plot 1 should plant crop 37 in season 4
Plot 1 should plant crop 43 in season 0
Plot 2 should plant crop 17 in season 0
Plot 

,作物类型,2024年1季产量（斤）,2024年2季产量（斤）,2025年1季产量（斤）,2025年2季产量（斤）,2026年1季产量（斤）,2026年2季产量（斤）,2027年1季产量（斤）,2027年2季产量（斤）,2028年1季产量（斤）,2028年2季产量（斤）,2029年1季产量（斤）,2029年2季产量（斤）,2030年1季产量（斤）,2030年2季产量（斤）,2031年1季产量（斤）,2031年2季产量（斤）,2032年1季产量（斤）,2032年2季产量（斤）
黄豆,粮食,5820.0,0.0,0.00,900.0,2000.00,0.0,7820.0,0.0,0.00,900.0,0.00,0.0,5960.00,1080.0,0.00,900.0,0.0,0.0
黑豆,粮食,25575.0,0.0,10000.00,0.0,15300.00,0.0,44125.0,0.0,6750.00,0.0,0.00,0.0,29000.00,0.0,10000.00,0.0,10750.0,0.0
红豆,粮食,9640.0,0.0,7980.00,0.0,9640.00,0.0,21790.0,0.0,4140.00,0.0,0.00,0.0,19400.00,0.0,0.00,0.0,6000.0,0.0
绿豆,粮食,8202.5,0.0,1650.00,0.0,2362.50,0.0,10400.0,1815.0,0.00,0.0,0.00,0.0,9687.50,0.0,0.00,0.0,2362.5,0.0
爬豆,粮食,14175.0,0.0,4542.50,0.0,7725.00,0.0,27720.0,0.0,0.00,0.0,0.00,0.0,27226.25,0.0,0.00,0.0,0.0,0.0
小麦,粮食,14400.0,0.0,20350.00,0.0,34750.00,0.0,22200.0,0.0,38200.00,0.0,16000.00,0.0,12920.00,0.0,40360.00,0.0,28960.0,0.0
玉米,粮食,289200.0,0.0,306300.00,0.0,292000.00,0.0,36200.0,0.0,32500.00,0.0,39000.00,0.0,55200.00,0.0,20000.00,0.0,35200.0,0.0
谷子,粮食,92730.0,0.0,88550.00,0.0,89580.00,0.0,10725.0,0.0,10450.00,0.0,10175.00,0.0,10950.00,0.0,5225.00,0.0,16175.0,0.0
高粱,粮食,17287.5,0.0,26287.50,0.0,9000.00,0.0,30000.0,0.0,30000.00,0.0,0.00,0.0,19350.00,0.0,38250.00,0.0,0.0,0.0
黍子,粮食,125750.0,0.0,140612.50,0.0,140612.50,0.0,33075.0,0.0,17625.00,0.0,0.00,0.0,31050.00,0.0,9450.00,0.0,7125.0,0.0


In [63]:
sale = sales.values #销量
df['平均成本'] = df['亩产量/斤']/df['种植成本/(元/亩)'] #平均成本
cost = df.sort_values(by='作物编号').drop_duplicates(subset='作物名称')['平均成本'].values
price = df['平均销售单价'].values

result = pd.DataFrame()
result.index=reward.index
for c in reward.columns[1:]:
    ls = []
    for i,o,n,m in zip(reward[c].values,sale,cost,price):
        if i > o:
            ls.append(o*(m-n)+(i-o)*(m-n)*0.5)
        else:
            ls.append(i*(m-n))
    result[c+'盈利'] = ls
result['作物类型'] = reward['作物类型']
result.groupby('作物类型').sum().to_excel('问题2\\情况一不同年份总的盈利情况.xlsx')
result.groupby('作物类型').sum()

,2024年1季产量（斤）盈利,2024年2季产量（斤）盈利,2025年1季产量（斤）盈利,2025年2季产量（斤）盈利,2026年1季产量（斤）盈利,2026年2季产量（斤）盈利,2027年1季产量（斤）盈利,2027年2季产量（斤）盈利,2028年1季产量（斤）盈利,2028年2季产量（斤）盈利,2029年1季产量（斤）盈利,2029年2季产量（斤）盈利,2030年1季产量（斤）盈利,2030年2季产量（斤）盈利,2031年1季产量（斤）盈利,2031年2季产量（斤）盈利,2032年1季产量（斤）盈利,2032年2季产量（斤）盈利
作物类型,,,,,,,,,,,,,,,,,,
粮食,1.667762e+06,0.000000,1.680020e+06,2025.000000,1.750425e+06,0.000000,903808.148279,11260.102941,715987.756247,2213.602941,346906.461913,0.000000,576771.550759,2430.000000,252537.089695,2025.000000,-78407.341658,188.602941
蔬菜,4.056169e+05,463234.253247,1.005007e+06,564046.980519,3.775125e+05,550546.980519,746063.438808,531630.527082,600245.442365,626115.527082,340652.016633,584249.348511,531716.513829,589290.810105,513822.187756,477665.810105,927878.851117,596945.105559
食用菌,0.000000e+00,72284.438042,0.000000e+00,33189.438042,0.000000e+00,33189.438042,0.000000,108906.744995,0.000000,91806.744995,0.000000,87021.744995,0.000000,113691.744995,0.000000,87021.744995,0.000000,87021.744995


### 情况二：超过部分按 2023 年销售价格的 50%降价出售。

In [68]:
import pulp
import random


# 54 块地，每块地都有特定的类型 (0: 平旱地, 1: 梯田, 2: 山坡, 3: 水浇地, 4: 普通大棚, 5: 智慧大棚)
num_plots = 54
num_crops = len(df['作物编号'].tolist())  # 125 种属于不同地块不同季节的作物
land_types =  df['作物编号'].tolist()# 0: 平旱地, 1: 梯田, 2: 山坡, 3: 水浇地, 4: 普通大棚, 5: 智慧大棚
plot_type = df1['地块类型'].tolist()  # 每个地块对应的地块类型

# 定义变量 x[i][j][t]，表示在第i块地是否种植第j种作物，t 表示第 t 季节
x = pulp.LpVariable.dicts("x", (range(num_plots), range(num_crops), range(6)), cat="Binary")

areas = df1['地块面积/亩'].tolist()  # 地块面积

# 定义作物集
grain_crops = ls1  # 粮食作物集
rice_crop = ls2  # 水稻
vegetable1_crops = ls3  # 第一季蔬菜集
vegetable2_crops = ls4  # 第二季蔬菜集
fungi_crop = ls5  # 食用菌
vegetable_crops = ls6  # 智慧大棚、普通大棚蔬菜集
total_set = ls1 + ls2 + ls3 + ls4 + ls5 + ls6
ex_grain_crops = [item for item in total_set if item not in grain_crops]  # 除粮食作物集剩余元素
ex_rice_crop = [item for item in total_set if item not in rice_crop]  # 水稻..
ex_vegetable1_crops = [item for item in total_set if item not in vegetable1_crops]  # 第一季蔬菜集
ex_vegetable2_crops = [item for item in total_set if item not in vegetable2_crops]  # 第二季蔬菜集
ex_fungi_crop = [item for item in total_set if item not in fungi_crop]  # 食用菌
ex_vegetable_crops = [item for item in total_set if item not in vegetable_crops]  # 智慧大棚、普通大棚蔬菜集

bean_crops = bean #大豆索引
time = 6

ex_grain_crops = [item for item in total_set if item not in grain_crops]  # 除粮食作物集剩余元素

def opt():
    # 定义问题
    model = pulp.LpProblem("Land_Type_Optimization", pulp.LpMaximize)
    # 目标函数：利润最大化
    model += pulp.lpSum([(prices[j] * yields[j] - costs[i][j]) * areas[i]*0.25 * x[i][j][t] 
                         for i in range(num_plots) for j in range(num_crops) for t in range(time)]) - function(sales_min1,sales_min2,sales_min3) 
                        #策略二

    # 地块类型约束
    for i in range(num_plots):
        if plot_type[i] in [0, 1, 2]:  # 平旱地、梯田、山坡
            for o in [0,2,4]:
                model += pulp.lpSum([x[i][j][o] for j in grain_crops]) ==4 # 每块地只能种一种作物
                model += pulp.lpSum([x[i][j][o+1] for j in range(num_crops)]) == 0  # 只能种一季，第二季不种


            model += pulp.lpSum([x[i][j][0]+x[i][j][2]+x[i][j][4] for j in bean_crops]) == 1 # 三年内必须有一次豆类

            for o in range(time):
                model += pulp.lpSum([x[i][j][o] for j in ex_grain_crops]) == 0  # 其余不能种

        elif plot_type[i] == 3:  # 水浇地
            for o in [0,2,4]:
                model += pulp.lpSum([x[i][45][o]]) + pulp.lpSum([x[i][j][o] for j in vegetable1_crops]) == 1  # 一季水稻或第一季蔬菜
                model += pulp.lpSum([x[i][j][o] for j in vegetable1_crops]) == pulp.lpSum([x[i][j][o+1] for j in vegetable2_crops])
                model += pulp.lpSum([x[i][45][o+1]]) == 0

                #model += pulp.lpSum([x[i][j][o+1] for j in vegetable2_crops]) + pulp.lpSum([x[i][j][o] for j in rice_crop]) == 1  # 如果第一季种菜，第二季只能种特定蔬菜

                model += pulp.lpSum([x[i][j][o+1] for j in ex_vegetable2_crops]) == 0  # 其余的不能种
                model += pulp.lpSum([x[i][j][o] for j in ex_vegetable1_crops]) == 0  # 其余的不能种



            model += pulp.lpSum([x[i][j][o] for j in bean_crops for o in range(time)]) == 1 # 三年内必须有一次豆类

        elif plot_type[i] == 4:  # 普通大棚
            for o in [0,2,4]:
                model += pulp.lpSum([x[i][j][o] for j in vegetable_crops]) == 4  # 一季蔬菜
                model += pulp.lpSum([x[i][j][o+1] for j in fungi_crop]) ==2  # 一季食用菌



                model += pulp.lpSum([x[i][j][o] for j in ex_vegetable_crops]) == 0  # 其余不能种
                model += pulp.lpSum([x[i][j][o+1] for j in ex_fungi_crop]) == 0  # 其余不能种

            model += pulp.lpSum([x[i][j][0]+x[i][j][2]+x[i][j][4] for j in bean_crops]) == 1 # 三年内必须有一次豆类

        elif plot_type[i] == 5:  # 智慧大棚
            for o in range(time):
                model += pulp.lpSum([x[i][j][o] for j in vegetable_crops]) == 4  # 一季蔬菜
                model += pulp.lpSum([x[i][j][o] for j in ex_vegetable_crops]) == 0  # 其余不能种



            model += pulp.lpSum([x[i][j][o] for j in bean_crops for o in range(time)]) == 1 # 三年内必须有一次豆类
    for j in range(num_crops):
        model += pulp.lpSum([x[i][j][o] for i in range(num_plots) for o in range(time)]) <= 2
        model += pulp.lpSum([x[i][j][o] for i in range(num_plots) if plot_type[i] == 4 for o in range(time)]) >= 2

    model.solve()

    # 输出结果
    for i in range(num_plots):
        for j in range(num_crops):
            for t in range(6):
                if pulp.value(x[i][j][t]) == 1:
                    print(f"Plot {i} should plant crop {j} in season {t}")
    return x

#制作结果图表
a = df2[['作物名称','作物类型']].drop_duplicates()
dic = {}
for i,o in zip(df2.作物名称,df2.作物类型):
    dic[i] = o               
reward = pd.DataFrame(index=df['作物名称'].unique())
reward['作物类型'] = [dic[i] for i in reward.index]

for wh in range(3):
    group()
    x = opt()
    for t in range(6):
        result = []
        product = []
        for i in range(num_plots):
            tem = []
            product_tem = []
            for j in range(num_crops):
                if pulp.value(x[i][j][t]) == 1:
                    tem.append(df1['地块面积/亩'].values[i])
                    product_tem.append(df1['地块面积/亩'].values[i]*df['亩产量/斤'].values[j]*0.25)
                else:
                    tem.append(0)
                    product_tem.append(0)
                    
            result.append(tem)
            product.append(product_tem)
        result = pd.DataFrame(result,index=df1['地块名称'].values,columns=df['作物编号'].values).T
        result = result.groupby(result.index).sum().T
        result.columns = df['作物名称'].unique()
        result.to_excel(f'问题2\\result1_2_{t%2+1}季(第{2023+t//2+1+wh*3}年).xlsx')
        
        product = pd.DataFrame(product,index=df1['地块名称'].values,columns=df['作物编号'].values).T
        product = product.groupby(product.index).sum().T
        product.columns = df['作物名称'].unique()
        reward[f'{2023+t//2+1+wh*3}年{t%2+1}季产量（斤）'] = product.sum()
reward

Plot 0 should plant crop 1 in season 0
Plot 0 should plant crop 6 in season 2
Plot 0 should plant crop 6 in season 4
Plot 0 should plant crop 9 in season 0
Plot 0 should plant crop 9 in season 4
Plot 0 should plant crop 14 in season 0
Plot 0 should plant crop 14 in season 2
Plot 0 should plant crop 14 in season 4
Plot 0 should plant crop 29 in season 0
Plot 0 should plant crop 29 in season 2
Plot 0 should plant crop 29 in season 4
Plot 0 should plant crop 41 in season 2
Plot 1 should plant crop 4 in season 2
Plot 1 should plant crop 7 in season 0
Plot 1 should plant crop 14 in season 0
Plot 1 should plant crop 14 in season 2
Plot 1 should plant crop 14 in season 4
Plot 1 should plant crop 22 in season 0
Plot 1 should plant crop 22 in season 4
Plot 1 should plant crop 28 in season 4
Plot 1 should plant crop 29 in season 0
Plot 1 should plant crop 29 in season 2
Plot 1 should plant crop 29 in season 4
Plot 1 should plant crop 37 in season 2
Plot 2 should plant crop 14 in season 0
Plot 2 

Plot 0 should plant crop 1 in season 4
Plot 0 should plant crop 6 in season 4
Plot 0 should plant crop 8 in season 0
Plot 0 should plant crop 8 in season 2
Plot 0 should plant crop 8 in season 4
Plot 0 should plant crop 26 in season 0
Plot 0 should plant crop 26 in season 2
Plot 0 should plant crop 36 in season 0
Plot 0 should plant crop 36 in season 2
Plot 0 should plant crop 36 in season 4
Plot 0 should plant crop 41 in season 0
Plot 0 should plant crop 41 in season 2
Plot 1 should plant crop 2 in season 4
Plot 1 should plant crop 7 in season 0
Plot 1 should plant crop 7 in season 2
Plot 1 should plant crop 7 in season 4
Plot 1 should plant crop 8 in season 0
Plot 1 should plant crop 8 in season 2
Plot 1 should plant crop 8 in season 4
Plot 1 should plant crop 13 in season 2
Plot 1 should plant crop 36 in season 0
Plot 1 should plant crop 36 in season 2
Plot 1 should plant crop 36 in season 4
Plot 1 should plant crop 39 in season 0
Plot 2 should plant crop 7 in season 0
Plot 2 should

Plot 0 should plant crop 1 in season 2
Plot 0 should plant crop 6 in season 2
Plot 0 should plant crop 6 in season 4
Plot 0 should plant crop 9 in season 0
Plot 0 should plant crop 9 in season 4
Plot 0 should plant crop 21 in season 0
Plot 0 should plant crop 21 in season 4
Plot 0 should plant crop 22 in season 0
Plot 0 should plant crop 22 in season 2
Plot 0 should plant crop 22 in season 4
Plot 0 should plant crop 36 in season 0
Plot 0 should plant crop 41 in season 2
Plot 1 should plant crop 2 in season 2
Plot 1 should plant crop 7 in season 0
Plot 1 should plant crop 14 in season 2
Plot 1 should plant crop 14 in season 4
Plot 1 should plant crop 22 in season 0
Plot 1 should plant crop 22 in season 2
Plot 1 should plant crop 22 in season 4
Plot 1 should plant crop 29 in season 0
Plot 1 should plant crop 40 in season 0
Plot 1 should plant crop 40 in season 2
Plot 1 should plant crop 40 in season 4
Plot 1 should plant crop 43 in season 4
Plot 2 should plant crop 22 in season 0
Plot 2 

,作物类型,2024年1季产量（斤）,2024年2季产量（斤）,2025年1季产量（斤）,2025年2季产量（斤）,2026年1季产量（斤）,2026年2季产量（斤）,2027年1季产量（斤）,2027年2季产量（斤）,2028年1季产量（斤）,2028年2季产量（斤）,2029年1季产量（斤）,2029年2季产量（斤）,2030年1季产量（斤）,2030年2季产量（斤）,2031年1季产量（斤）,2031年2季产量（斤）,2032年1季产量（斤）,2032年2季产量（斤）
黄豆,粮食,7655.00,0.0,0.0,2000.0,0.0,0.0,3000.00,950.0,1235.00,0.0,0.00,0.0,8360.00,0.0,0.00,0.0,0.00,0.0
黑豆,粮食,28400.00,0.0,8550.0,0.0,15725.0,0.0,18825.00,0.0,22050.00,0.0,10000.00,0.0,23375.00,0.0,16750.00,0.0,10750.00,0.0
红豆,粮食,6710.00,0.0,12705.0,0.0,6000.0,0.0,3800.00,0.0,3150.00,0.0,14150.00,0.0,16550.00,0.0,5500.00,0.0,0.00,0.0
绿豆,粮食,13352.50,0.0,0.0,0.0,0.0,0.0,7950.00,0.0,3935.00,0.0,0.00,0.0,13331.25,1650.0,0.00,0.0,0.00,0.0
爬豆,粮食,13451.25,0.0,16100.0,0.0,0.0,0.0,10051.25,0.0,12481.25,0.0,5187.50,0.0,19556.25,0.0,8756.25,0.0,0.00,0.0
小麦,粮食,12240.00,0.0,37240.0,0.0,25800.0,0.0,34850.00,0.0,13600.00,0.0,23640.00,0.0,14400.00,0.0,24360.00,0.0,38760.00,0.0
玉米,粮食,33300.00,0.0,36200.0,0.0,37100.0,0.0,262800.00,0.0,245700.00,0.0,300800.00,0.0,37000.00,0.0,36200.00,0.0,39000.00,0.0
谷子,粮食,10725.00,0.0,9900.0,0.0,10725.0,0.0,88635.00,0.0,93040.00,0.0,99840.00,0.0,106470.00,0.0,101070.00,0.0,100970.00,0.0
高粱,粮食,18450.00,0.0,17550.0,0.0,18000.0,0.0,167445.00,0.0,174120.00,0.0,167445.00,0.0,19650.00,0.0,19650.00,0.0,17100.00,0.0
黍子,粮食,25500.00,0.0,0.0,0.0,23562.5,0.0,15981.25,0.0,17950.00,0.0,15031.25,0.0,17625.00,0.0,8500.00,0.0,27075.00,0.0


In [70]:
sale = sales.values #销量
df['平均成本'] = df['亩产量/斤']/df['种植成本/(元/亩)'] #平均成本
cost = df.sort_values(by='作物编号').drop_duplicates(subset='作物名称')['平均成本'].values
price = df['平均销售单价'].values

result = pd.DataFrame()
result.index=reward.index
for c in reward.columns[1:]:
    ls = []
    for i,o,n,m in zip(reward[c].values,sale,cost,price):
        if i > o:
            ls.append(o*(m-n)+(i-o)*(m-n)*0.5)
        else:
            ls.append(i*(m-n))
    result[c+'盈利'] = ls
result['作物类型'] = reward['作物类型']
result.groupby('作物类型').sum().to_excel('问题2\\情况二不同年份总的盈利情况.xlsx')
result.groupby('作物类型').sum()

,2024年1季产量（斤）盈利,2024年2季产量（斤）盈利,2025年1季产量（斤）盈利,2025年2季产量（斤）盈利,2026年1季产量（斤）盈利,2026年2季产量（斤）盈利,2027年1季产量（斤）盈利,2027年2季产量（斤）盈利,2028年1季产量（斤）盈利,2028年2季产量（斤）盈利,2029年1季产量（斤）盈利,2029年2季产量（斤）盈利,2030年1季产量（斤）盈利,2030年2季产量（斤）盈利,2031年1季产量（斤）盈利,2031年2季产量（斤）盈利,2032年1季产量（斤）盈利,2032年2季产量（斤）盈利
作物类型,,,,,,,,,,,,,,,,,,
粮食,1.218938e+06,0.000000,814274.382147,4500.000000,8.272791e+05,0.000000,1.670819e+06,2137.500000,1.612790e+06,0.000000,1.683878e+06,0.000000,1.685965e+06,10065.000000,1.478534e+06,0.000000,1.573080e+06,0.000000
蔬菜,5.058928e+05,-206176.564599,879457.130114,-206176.564599,1.582848e+06,-435551.564599,8.996960e+05,-219676.564599,5.193098e+05,-206176.564599,9.315509e+05,-422051.564599,9.856678e+05,588635.935401,5.318531e+05,490510.935401,7.570224e+05,575135.935401
食用菌,0.000000e+00,53494.085956,0.000000,53399.085956,0.000000e+00,41524.085956,0.000000e+00,96422.704819,0.000000e+00,95402.704819,0.000000e+00,95402.704819,0.000000e+00,149073.618863,0.000000e+00,148713.618863,0.000000e+00,148713.618863
